In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import time
from datasets import Dataset, DatasetDict, load_dataset
import torchtext
import tqdm
import evaluate
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Using", device, "device")
torch.cuda.empty_cache()
import os
import sys

Using cuda device


In [5]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

## SPM Tokeniser for Python ver < 3.10

In [13]:
# sys.path.append(os.path.abspath('../../tokenisation/sentencepiece_custom'))
en_model_absolute_path = os.path.abspath('../../tokenisation/sentencepiece_custom/en.model')
zh_model_absolute_path = os.path.abspath('../../tokenisation/sentencepiece_custom/zh.model')

In [14]:
from tokeniser_lt_310 import BaseBPETokeniser

tokenizer = BaseBPETokeniser(en_model_file=en_model_absolute_path, zh_model_file=zh_model_absolute_path)

# If without padding
encoded_en_no_padding = tokenizer.encode_en_no_padding("This is", max_len=3)
print("Encoded en:", encoded_en_no_padding)
encoded_zh_no_padding = tokenizer.encode_zh_no_padding("这是", max_len=3)
print("Encoded zh:", encoded_zh_no_padding)

# Decoding (same) 
decoded_en = tokenizer.decode_src(encoded_en_no_padding)
decoded_zh = tokenizer.decode(encoded_zh_no_padding)

decoded_en, decoded_zh

Encoded en: [308, 56]
Encoded zh: [32]


('This is', '这是')

## Get Data

In [ ]:
dataset = load_dataset("iwslt2017", "iwslt2017-en-zh")
train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"],
)

# first 10k rows
train_data = train_data.select(range(10000))
train_data

Dataset({
    features: ['translation'],
    num_rows: 10000
})

## Tokenise

In [ ]:
unk_g, pad_g, bos_g, eos_g = tokenizer.get_special_ids(lang='zh')
max_length_g = 285

def tokenize_example(example, bos_token_id=bos_g, eos_token_id=eos_g):
    en_tokens = tokenizer.encode_en_no_padding(example['translation']["en"], max_len=max_length_g)
    zh_tokens = tokenizer.encode_zh_no_padding(example['translation']["zh"], max_len=max_length_g)
    en_tokens = [bos_token_id] + en_tokens + [eos_token_id]
    zh_tokens = [bos_token_id] + zh_tokens + [eos_token_id]
    return {"en_ids": en_tokens, "zh_ids": zh_tokens}

train_data = train_data.map(tokenize_example)
valid_data = valid_data.map(tokenize_example)
test_data = test_data.map(tokenize_example)

train_data

Map: 100%|██████████| 8549/8549 [00:00<00:00, 10507.80 examples/s]


Dataset({
    features: ['translation', 'en_ids', 'zh_ids'],
    num_rows: 10000
})

In [ ]:
# Check bos & eos
train_data[0]['en_ids'][0] ,  train_data[0]['en_ids'][-1] , train_data[0]['zh_ids'][0] ,  train_data[0]['zh_ids'][-1]

(1, 2, 1, 2)

In [ ]:
# Convert to tensors
data_type = "torch"
format_columns = ["en_ids", "zh_ids"]

train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

## Dataloaders

In [ ]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_zh_ids = [example["zh_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_zh_ids = nn.utils.rnn.pad_sequence(batch_zh_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "zh_ids": batch_zh_ids,
        }
        return batch

    return collate_fn

def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [ ]:
batch_size = 16
pad_index = 3

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

## Architecture

In [ ]:
class Encoder(nn.Module):
    def __init__(
        self, input_dim, embedding_dim, encoder_hidden_dim, decoder_hidden_dim, dropout
    ):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, encoder_hidden_dim, bidirectional=True)
        self.fc = nn.Linear(encoder_hidden_dim * 2, decoder_hidden_dim) # encoder_hidden  * 2 if bidirectional!
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, hidden = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        # outputs are always from the last layer
        # hidden [-2, :, : ] is the last of the forwards RNN
        # hidden [-1, :, : ] is the last of the backwards RNN
        # initial decoder hidden is final hidden state of the forwards and backwards
        # encoder RNNs fed through a linear layer
        hidden = torch.tanh(
            self.fc(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        )
        # hidden = torch.tanh(self.fc(hidden[-1, :, :])) # IF NOT BIDIRECTIONAL
        # outputs = [src length, batch size, encoder hidden dim * 2]
        # hidden = [batch size, decoder hidden dim]
        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, encoder_hidden_dim, decoder_hidden_dim):
        super().__init__()
        self.attn_fc = nn.Linear(
            (encoder_hidden_dim * 2) + decoder_hidden_dim, decoder_hidden_dim # IF BIDIRECTIONAL MUST encoder_hidden_dim * 2 
        )
        self.v_fc = nn.Linear(decoder_hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden = [batch size, decoder hidden dim]
        # encoder_outputs = [src length, batch size, encoder hidden dim * 2]
        batch_size = encoder_outputs.shape[1]
        src_length = encoder_outputs.shape[0]
        # repeat decoder hidden state src_length times
        hidden = hidden.unsqueeze(1).repeat(1, src_length, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # hidden = [batch size, src length, decoder hidden dim]
        # encoder_outputs = [batch size, src length, encoder hidden dim * 2]
        energy = torch.tanh(self.attn_fc(torch.cat((hidden, encoder_outputs), dim=2)))
        # energy = [batch size, src length, decoder hidden dim]
        attention = self.v_fc(energy).squeeze(2)
        # attention = [batch size, src length]
        return torch.softmax(attention, dim=1)

In [ ]:
class Decoder(nn.Module):
    def __init__(
        self,
        output_dim,
        embedding_dim,
        encoder_hidden_dim,
        decoder_hidden_dim,
        dropout,
        attention,
    ):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.GRU((encoder_hidden_dim * 2) + embedding_dim, decoder_hidden_dim) # IF BIDIRECTIONAL MUST encoder_hidden_dim * 2 
        self.fc_out = nn.Linear(
            (encoder_hidden_dim * 2) + decoder_hidden_dim + embedding_dim, output_dim # IF BIDIRECTIONAL MUST encoder_hidden_dim * 2 
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        # input = [batch size]
        # hidden = [batch size, decoder hidden dim]
        # encoder_outputs = [src length, batch size, encoder hidden dim * 2]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        a = self.attention(hidden, encoder_outputs)
        # a = [batch size, src length]
        a = a.unsqueeze(1)
        # a = [batch size, 1, src length]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # encoder_outputs = [batch size, src length, encoder hidden dim * 2]
        weighted = torch.bmm(a, encoder_outputs)
        # weighted = [batch size, 1, encoder hidden dim * 2]
        weighted = weighted.permute(1, 0, 2)
        # weighted = [1, batch size, encoder hidden dim * 2]
        rnn_input = torch.cat((embedded, weighted), dim=2)
        # rnn_input = [1, batch size, (encoder hidden dim * 2) + embedding dim]
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        # output = [seq length, batch size, decoder hid dim * n directions]
        # hidden = [n layers * n directions, batch size, decoder hid dim]
        # seq len, n layers and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, decoder hidden dim]
        # hidden = [1, batch size, decoder hidden dim]
        # this also means that output == hidden
        assert (output == hidden).all()
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))
        # prediction = [batch size, output dim]
        return prediction, hidden.squeeze(0), a.squeeze(1)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        batch_size = src.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # encoder_outputs is all hidden states of the input sequence, back and forwards
        # hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
        # outputs = [src length, batch size, encoder hidden dim * 2]
        # hidden = [batch size, decoder hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden state and all encoder hidden states
            # receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, decoder hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [ ]:
input_dim = len(tokenizer) # 16384
output_dim = len(tokenizer)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
encoder_hidden_dim = 512
decoder_hidden_dim = 512
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

attention = Attention(encoder_hidden_dim, decoder_hidden_dim)

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    decoder_dropout,
    attention,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if "weight" in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(16384, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn_fc): Linear(in_features=1536, out_features=512, bias=True)
      (v_fc): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(16384, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=16384, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 44,198,400 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_g)

In [ ]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["en_ids"].to(device)
        trg = batch["zh_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["en_ids"].to(device)
            trg = batch["zh_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

## Training

In [ ]:
n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5
CHECKPOINT_DIR = "./model_checkpoints"
checkpoint_path = f"{CHECKPOINT_DIR}/model_checkpoint.pt"
if not os.path.exists(CHECKPOINT_DIR):
    os.makedirs(CHECKPOINT_DIR)

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save({'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': best_valid_loss}, f"{CHECKPOINT_DIR}/model_checkpoint.pt")
        print(f"Checkpoint saved at epoch {epoch+1}")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

 10%|█         | 1/10 [04:11<37:41, 251.27s/it]

Checkpoint saved at epoch 1
	Train Loss:   7.806 | Train PPL: 2455.526
	Valid Loss:   7.963 | Valid PPL: 2873.110


 20%|██        | 2/10 [08:47<35:29, 266.16s/it]

Checkpoint saved at epoch 2
	Train Loss:   7.278 | Train PPL: 1448.205
	Valid Loss:   7.882 | Valid PPL: 2648.255


 30%|███       | 3/10 [13:26<31:42, 271.83s/it]

...

	Train Loss:   3.096 | Train PPL:  22.116
	Valid Loss:   9.442 | Valid PPL: 12608.620


 90%|█████████ | 9/10 [40:51<04:31, 271.97s/it]

	Train Loss:   2.760 | Train PPL:  15.797
	Valid Loss:   9.593 | Valid PPL: 14660.283


100%|██████████| 10/10 [45:10<00:00, 271.01s/it]

	Train Loss:   2.481 | Train PPL:  11.953
	Valid Loss:   9.774 | Valid PPL: 17578.213


## Train from best state

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time % 60)
    return elapsed_mins, elapsed_secs

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time % 60)
    return elapsed_mins, elapsed_secs

patience = 2  
no_improvement_epochs = 0

if os.path.exists(checkpoint_path):
    print("Loading checkpoint...")
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    best_valid_loss = checkpoint['loss'] 
    print(f"Current best val loss : {best_valid_loss}")
else:
    print("Training started, no checkpoint found.")

for epoch in tqdm.tqdm(range(n_epochs)):
    start_time = time.time()
    train_loss = train_fn(model, train_data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device)
    valid_loss = evaluate_fn(model, valid_data_loader, criterion, device)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save({'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': best_valid_loss}, f"{CHECKPOINT_DIR}/model_checkpoint.pt")
        print(f"Checkpoint saved at epoch {epoch+1}")
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1
        
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {np.exp(train_loss):7.3f}')
    print(f'\t Valid Loss: {valid_loss:.3f} | Valid PPL: {np.exp(valid_loss):7.3f}')
    # Auto stop
    if no_improvement_epochs >= patience:
        print(f"No improvement in val loss for {patience} consecutive epochs. Stopping training.")
        break
            
print("Training completed.")

Loading checkpoint...
Current best val loss : 7.84190318367698


 10%|█         | 1/10 [04:26<40:01, 266.88s/it]

Epoch: 01 | Time: 4m 26s
	Train Loss: 6.163 | Train PPL: 475.007
	 Valid Loss: 7.988 | Valid PPL: 2946.453


 10%|█         | 1/10 [08:55<1:20:16, 535.17s/it]

Epoch: 02 | Time: 4m 28s
	Train Loss: 5.195 | Train PPL: 180.304
	 Valid Loss: 8.337 | Valid PPL: 4176.944
No improvement in val loss for 2 consecutive epochs. Stopping training.
Training completed.


## Eval

In [ ]:
checkpoint = torch.load(r"model_checkpoints\model_checkpoint.pt")
if 'model_state_dict' in checkpoint:
    model.load_state_dict(checkpoint['model_state_dict'])  
else:
    model.load_state_dict(checkpoint) 

test_loss = evaluate_fn(model, test_data_loader, criterion, device)
print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

| Test Loss: 7.481 | Test PPL: 1773.180 |


In [ ]:
def translate_sentence(
    sentence,
    model,
    # lower,
    device,
    max_output_length=max_length_g,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            en_tokens = tokenizer.encode_en_no_padding(sentence,max_len=max_length_g)
        else:
            # en_tokens = [token for token in sentence]
            print('sentence is not an instance!')
        ids = [1] + en_tokens + [2]
        # ids = en_vocab.lookup_indices(en_tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        encoder_outputs, hidden = model.encoder(tensor)
        inputs = [1]
        attentions = torch.zeros(max_output_length, 1, len(ids))
        for i in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, attention = model.decoder(
                inputs_tensor, hidden, encoder_outputs
            )
            attentions[i] = attention
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == 2:
                break
        zh_tokens = tokenizer.decode(inputs)
        translated = ' '.join(zh_tokens)
    return translated, en_tokens, attentions[: len(inputs) - 1]

In [ ]:
def plot_attention(sentence, translation, attention):
    fig, ax = plt.subplots(figsize=(10, 10))
    attention = attention.squeeze(1).numpy()
    cax = ax.matshow(attention, cmap="bone")
    ax.set_xticks(ticks=np.arange(len(sentence)), labels=sentence, rotation=90, size=15)
    translation = translation[1:]
    ax.set_yticks(ticks=np.arange(len(translation)), labels=translation, size=15)
    plt.show()
    plt.close()

In [ ]:
sentence = test_data[2]['translation']["en"]
expected_translation = test_data[2]['translation']["zh"]

sentence, expected_translation

('The marshmallow has to be on top.', '棉花糖必须放在最上面')

In [ ]:
translation, sentence_tokens, attention = translate_sentence(
    sentence,
    model,
    device,
)
translation

'  ⁇     ⁇   。'

In [ ]:
sentence_tokens

[153, 15516, 299, 25, 58, 70, 1097, 16259]

In [ ]:
# plot_attention(sentence_tokens, translation, attention)

In [ ]:
test_data[0]['translation']

{'en': 'Several years ago here at TED, Peter Skillman  introduced a design challenge  called the marshmallow challenge.',
 'zh': '几年前，在TED大会上， Peter Skillman 介绍了一个设计挑战 叫做“棉花糖挑战”'}

In [ ]:
translation, sentence_tokens, attention = translate_sentence(sentence,model,device)
en = [] 
zh = [] 
pred = []

import pandas as pd

for i in range(0,8549):
    en.append(test_data[i]['translation']['en'])
    zh.append(test_data[i]['translation']['zh'])
    t,s,a = translate_sentence(test_data[i]['translation']['en'],model,device)
    pred.append(t)

df = pd.DataFrame()
df['en'] = en
df['zh'] = zh 
df['pred'] = pred

df.to_csv("060424_0000_model_checkpoint.csv")

## Ignore

In [ ]:
# checkpoint_path = f"{CHECKPOINT_DIR}/060424_0000_model_checkpoint.pt"

# if os.path.exists(checkpoint_path):
#     print("Loading checkpoint...")
#     checkpoint = torch.load(checkpoint_path)
#     print(f"1600_model val loss : {checkpoint['loss']}")



Loading checkpoint...
1600_model val loss : 7.873250102996826
